In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [5]:
# import torch_xla
# import torch_xla.core.xla_model as xm

In [6]:
from gcn.model import GCN
from gcn.utils import *
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import json
from collections import OrderedDict

### Settings

In [7]:
model_name = 'gcn'
dataset = 'arx' # 'citeseer', 'cora', 'pubmed', 'arx'
task = 'classification' # 'classification', 'link_prediction'
feat_norm = False
n_iter = 10

In [8]:
epochs = 200
learning_rate = 0.01
weight_decay = 5e-4
num_hidden = 32
dropout = 0.5
train_ratio = 0.5
val_ratio = train_ratio * 0.1 # len_val = len_train * val_ratio
train_val_ratio = [train_ratio, val_ratio]
n_diff_range = [1,2,3,4,5]
alpha_range = [0.1, 0.3, 0.5, 0.7, 0.9]

In [9]:
# n_diff = 3
# alpha = 0.5

In [10]:
# hparams = epochs, learning_rate, weight_decay, num_hidden, dropout, train_val_ratio, n_diff, alpha

### Import dataset

In [11]:
if task == 'link_prediction':
    adj, feature = load_data(dataset, task, feat_norm)
elif task == 'classification':
    adj, feature, labels = load_data(dataset, task, feat_norm)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = xm.xla_device()
device

device(type='cuda')

### GCN(iterative operation)

In [ ]:
for n_diff in n_diff_range:
    for alpha in alpha_range:
        
        hparams = epochs, learning_rate, weight_decay, num_hidden, dropout, train_val_ratio, n_diff, alpha

        results = gcn_iter(adj, feature, labels, hparams, n_iter=n_iter)

        date = (datetime.now() + timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")
        hparams_add = hparams[:-2] + (feat_norm,)
        if n_diff == 1 and alpha == 0.5:
            experiment_mode = 'normal mode(n_diff={}, alpha={})'.format(n_diff, alpha)
        else:
            experiment_mode = 'simplified diffusion mode(n_diff={}, alpha={})'.format(n_diff, alpha)
        
        log_dict= OrderedDict()
        log_dict['model'] = model_name
        log_dict['dataset'] = dataset
        log_dict['datetime'] = date
        log_dict['simplified diffusion'] =  experiment_mode
        log_dict['setting_order'] = ['epochs', 'lr', 'weight_decay', 'num_hidden', 'dropout', 'train_val_ratio', 'feat_norm']
        log_dict['setting_value'] = hparams_add
        log_dict['iteration'] = n_iter
        log_dict['acc'] = results[0]
        log_dict['acc_mean'] = np.mean(results[0])
        log_dict['acc_std'] = np.std(results[0])
        log_dict['loss'] = results[1]
        log_dict['loss_mean'] = np.mean(results[1])
        log_dict['loss_std'] = np.std(results[1])
        log_dict['elapsed time(s)'] = np.round(results[2], 4)
        print(pd.DataFrame(log_dict.items(), columns=['key', 'value']))

        data = json.load(open('results_classification.json'))
        data.append(log_dict)
        with open('results_classification.json', 'w') as f:
            json.dump(data, f)
        print("Last data saved at: {}".format((datetime.now() + timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")))
        print("Total data num: {}".format(len(data)))

Dataset: arx
Simdiff_params: n_diff =  1, alpha = 0.1
Start time: 2021-09-21 05:54:50
Experiment 1 - 1/200 val_acc: 0.2680 val_loss: 1.9894
Experiment 1 - 51/200 val_acc: 0.7720 val_loss: 0.7084
Experiment 1 - 101/200 val_acc: 0.7680 val_loss: 0.6963
Experiment 1 - 151/200 val_acc: 0.7600 val_loss: 0.7103
Experiment 1 result - test_acc: 0.7112, test_loss: 0.9225
Experiment 2 - 1/200 val_acc: 0.0960 val_loss: 2.0649
Experiment 2 - 51/200 val_acc: 0.7280 val_loss: 0.8315
Experiment 2 - 101/200 val_acc: 0.6880 val_loss: 0.8349
Experiment 2 - 151/200 val_acc: 0.7040 val_loss: 0.8562
Experiment 2 result - test_acc: 0.7244, test_loss: 0.8806
Experiment 3 - 1/200 val_acc: 0.2400 val_loss: 1.9969
Experiment 3 - 51/200 val_acc: 0.7280 val_loss: 0.8093
Experiment 3 - 101/200 val_acc: 0.7400 val_loss: 0.8196
Experiment 3 - 151/200 val_acc: 0.7320 val_loss: 0.8516
Experiment 3 result - test_acc: 0.7204, test_loss: 0.8863
Experiment 4 - 1/200 val_acc: 0.0720 val_loss: 2.1281
Experiment 4 - 51/200 v

### GCN(single operation)

In [ ]:
# results = gcn_iter(adj, feature, labels, hparams, n_iter=n_iter)

### Save log

In [ ]:
# date = (datetime.now() + timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")
# hparams_add = hparams[:-2] + (feat_norm,)
# if n_diff == 1 and alpha == 0.5:
#     experiment_mode = 'normal mode(n_diff={}, alpha={})'.format(n_diff, alpha)
# else:
#     experiment_mode = 'simplified diffusion mode(n_diff={}, alpha={})'.format(n_diff, alpha)

In [ ]:
# log_dict= OrderedDict()

In [ ]:
# log_dict['model'] = model_name
# log_dict['dataset'] = dataset
# log_dict['date'] = date
# log_dict['simplified diffusion'] =  experiment_mode
# log_dict['setting_order'] = ['epochs', 'lr', 'weight_decay', 'num_hidden', 'dropout', 'train_val_ratio', 'feat_norm']
# log_dict['setting_value'] = hparams_add
# log_dict['iteration'] = n_iter
# log_dict['acc'] = results[0]
# log_dict['acc_mean'] = np.mean(results[0])
# log_dict['acc_std'] = np.std(results[0])
# log_dict['loss'] = results[1]
# log_dict['loss_mean'] = np.mean(results[1])
# log_dict['loss_std'] = np.std(results[1])
# log_dict['elapsed time(s)'] = np.round(results[2], 4)
# log_dict

In [ ]:
# data = json.load(open('results_classification.json'))
# data.append(log_dict)

In [ ]:
# with open('results_classification.json', 'w') as f:
#     json.dump(data, f)

### Define GCN

In [13]:
def gcn_iter(adj, feature, labels, hparams, n_iter=1):
    
    epochs, learning_rate, weight_decay, num_hidden, dropout, train_val_ratio, n_diff, alpha = hparams
    len_train= int(len(feature)*train_val_ratio[0])       
    len_val= int(len_train*train_val_ratio[1])
    len_test= len(feature)-(len_train+len_val)
    
    acc_results = []
    loss_results = []
    start = time.time()
    print("Dataset: {}".format(dataset))
    print("Simdiff_params: n_diff =  {}, alpha = {}".format(n_diff, alpha))
    print("Start time: {}".format((datetime.now() + timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")))
    
    adj = preprocess_graph_diff(adj, n_diff, alpha)
    
    for i in range(n_iter):
        idx_train, idx_val, idx_test = split(len(feature), len_train, len_val, len_test)
        adj, feature, labels = adj.to(device), feature.to(device), labels.to(device)
        model = GCN(nfeat=feature.shape[1], nhid=num_hidden, nclass=labels.max().item()+1, dropout=dropout)
        model= model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(feature, adj)
            train_loss = F.nll_loss(output[idx_train], labels[idx_train])
            train_acc = accuracy(output[idx_train], labels[idx_train])
            train_loss.backward()
            optimizer.step()
    
            model.eval()
            output = model(feature, adj)
            val_loss = F.nll_loss(output[idx_val], labels[idx_val])
            val_acc = accuracy(output[idx_val], labels[idx_val])
            
            if epoch % 50 == 0:
                print('Experiment {} - {}/{} val_acc: {:.4f} val_loss: {:.4f}'.format(\
                      i+1, epoch+1, epochs, val_acc, val_loss), end='\n', flush=False)
        model.eval()
        output = model(feature, adj)
        test_acc = accuracy(output[idx_test], labels[idx_test])
        test_loss = F.nll_loss(output[idx_test], labels[idx_test])
        acc_results.append(float(test_acc.detach().cpu().numpy()))
        loss_results.append(float(test_loss.detach().cpu().numpy()))
        
        print('Experiment {} result - test_acc: {:.4f}, test_loss: {:.4f}'.format(i+1, test_acc, test_loss), end='\n')
    elapsed_time = time.time()-start    
    print('All experiments finished!', '\nElapsed time: {:.2f}s'.format(elapsed_time), end='\n')
    return acc_results, loss_results, elapsed_time

In [ ]:
# # if wanna clear results
# empty = []
# with open('results_classification.json', 'w') as f:
#     json.dump(empty, f)